# Part B — Sentiment Analysis Prompt Evaluation

Runs sentiment Prompt V1 and improved Prompt V2 on test emails.


In [2]:
import re, json
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

EMAILS_PATH = "data/sentiment_test_emails.txt"

def read_emails(path):
    with open(path, "r", encoding="utf-8") as f:
        text = f.read().strip()
    return [b.strip() for b in re.split(r"\n\s*\n", text) if b.strip()]

emails = read_emails(EMAILS_PATH)
emails


['Hi team,\nI am extremely unhappy that my issue is still not solved after 5 days.\nPlease fix it urgently.',
 'The product works really well!\nThank you for your amazing support.',
 'I’m confused about how to configure automations.\nCan you please guide me?',
 'This is very frustrating. I asked twice for a refund but got no response.',
 'Thanks a lot for the quick help — everything is working perfectly now.',
 'Our CSAT scores are not showing. Not sure if it is a bug or something we misconfigured.',
 'Things worked yesterday but today nothing is syncing. Quite annoying.',
 'Perfect! The setup works smoothly now.\nAppreciate it.',
 'We still can’t login even after resetting our password.\nPlease resolve this.',
 'Thank you for your support, but the emails are still not syncing properly.']

In [3]:
prompt_v1 = """
You are a sentiment classifier for customer-support emails.

Return a JSON object with:
- sentiment: positive, neutral, or negative
- confidence: a float between 0 and 1
- internal_reasoning: short explanation (for debugging)

Email:
{email}

Return ONLY valid JSON.
"""


In [4]:
def classify(email, prompt_template):
    prompt = prompt_template.format(email=email)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )

    raw = response.choices[0].message.content
    
    try:
        return json.loads(raw)
    except:
        return {"raw_output": raw}


In [5]:
results_v1 = []

for e in emails:
    res = classify(e, prompt_v1)
    results_v1.append(res)
    print(res)


{'sentiment': 'negative', 'confidence': 0.95, 'internal_reasoning': 'The customer expresses extreme unhappiness and urgency regarding an unresolved issue, indicating a strong negative sentiment.'}
{'sentiment': 'positive', 'confidence': 0.95, 'internal_reasoning': 'The email expresses satisfaction with the product and gratitude for support, indicating a positive sentiment.'}
{'raw_output': '```json\n{\n  "sentiment": "neutral",\n  "confidence": 0.85,\n  "internal_reasoning": "The email expresses confusion and requests guidance, indicating a neutral sentiment without strong positive or negative emotions."\n}\n```'}
{'sentiment': 'negative', 'confidence': 0.95, 'internal_reasoning': 'The email expresses frustration and dissatisfaction due to a lack of response regarding a refund request, indicating a negative sentiment.'}
{'sentiment': 'positive', 'confidence': 0.95, 'internal_reasoning': 'The customer expresses gratitude for the quick help and confirms that everything is working perfect

In [9]:
prompt_v2 = """
You are a strict sentiment classifier for customer-support emails.

Rules:
1. Never return markdown or codeblocks.
2. Return ONLY valid JSON.
3. Frustration, urgency, or anger => negative.
4. Appreciation or satisfaction => positive.
5. Questions without emotion => neutral.
6. Conflicting emotions => neutral.

Return ONLY:
{{
  "sentiment": "...",
  "confidence": 0.0,
  "internal_reasoning": "..."
}}

Email:
{email}
"""


In [10]:
results_v2 = []

for e in emails:
    res = classify(e, prompt_v2)
    results_v2.append(res)
    print(res)


{'sentiment': 'negative', 'confidence': 0.95, 'internal_reasoning': 'The email expresses extreme unhappiness and urgency regarding an unresolved issue, indicating frustration and a negative sentiment.'}
{'sentiment': 'positive', 'confidence': 0.95, 'internal_reasoning': 'The email expresses appreciation for the product and satisfaction with the support received, indicating a positive sentiment.'}
{'sentiment': 'neutral', 'confidence': 0.0, 'internal_reasoning': 'The email expresses a question without any emotional content, indicating confusion but not frustration or urgency.'}
{'sentiment': 'negative', 'confidence': 0.95, 'internal_reasoning': 'The email expresses frustration and indicates a lack of response regarding a refund request, which aligns with negative sentiment.'}
{'sentiment': 'positive', 'confidence': 0.95, 'internal_reasoning': 'The email expresses appreciation for the quick help and satisfaction with the outcome, indicating a positive sentiment.'}
{'sentiment': 'neutral'